In [1]:
pip install watchdog pandas


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import time
import sqlite3
import pandas as pd
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Folder to watch
WATCH_FOLDER = "watch_folder"

# Step 1: Create the watch_folder if it doesn't exist
def create_watch_folder():
    if not os.path.exists(WATCH_FOLDER):
        os.makedirs(WATCH_FOLDER)
        print(f"[✓] Created folder: {WATCH_FOLDER}")
    else:
        print(f"[✓] Folder already exists: {WATCH_FOLDER}")

# Step 2: Create database and table
def create_database():
    conn = sqlite3.connect("sales.db")
    conn.execute('''
        CREATE TABLE IF NOT EXISTS sales (
            OrderID INTEGER,
            CustomerName TEXT,
            Product TEXT,
            Quantity INTEGER,
            Price REAL,
            OrderDate TEXT
        )
    ''')
    conn.commit()
    conn.close()
    print("[✓] Database and table ready.")

# Step 3: Insert CSV data into database
def insert_data_from_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        conn = sqlite3.connect("sales.db")
        df.to_sql("sales", conn, if_exists="append", index=False)
        conn.close()
        print(f"[✓] Data inserted from: {file_path}")
    except Exception as e:
        print(f"[X] Error reading file: {e}")

# Step 4: Handle new file creation
class WatchHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.src_path.endswith(".csv"):
            print(f"[+] New CSV detected: {event.src_path}")
            time.sleep(1)  # Wait a moment to let file finish copying
            insert_data_from_csv(event.src_path)

# Step 5: Start watching the folder
def start_watching():
    observer = Observer()
    event_handler = WatchHandler()
    observer.schedule(event_handler, WATCH_FOLDER, recursive=False)
    observer.start()
    print(f"👀 Watching folder: {WATCH_FOLDER} for new CSV files...")

    try:
        while True:
            time.sleep(1)  # Keeps the script running
    except KeyboardInterrupt:
        observer.stop()
        print("\n[!] Stopped watching.")
    observer.join()

# Run the steps
if __name__ == "__main__":
    create_watch_folder()
    create_database()
    start_watching()


[✓] Folder already exists: watch_folder
[✓] Database and table ready.
👀 Watching folder: watch_folder for new CSV files...


In [ ]:
python file_listener.py

In [ ]:
OrderID,CustomerName,Product,Quantity,Price,OrderDate
1001,Alice,Notebook,2,200,2023-01-12
1002,Bob,Pen,5,25,2023-01-13
1003,Charlie,Marker,3,30,2023-01-13
1004,David,Notebook,1,200,2023-01-14
1005,Eva,Pen,10,25,2023-01-15
